In [ ]:
import os
from PIL import Image

import h5py
import numpy as np

from db_helper import get_files_paths_recursive

In [ ]:
path = R"C:\Users\Marcin\Dysk Google\masterDB\fake"
filename = R"C:\Users\Marcin\Dysk Google\masterDB\fake.h5"
dataset_name = 'fake'

In [ ]:
def load_data_h5(path, dataset_name):
    with h5py.File(path, "r") as hf:
        X = hf[dataset_name][:]
        hf.close()
    return X

In [ ]:
def convert_dataset_to_h5(db_path, h5_path, dataset_name, img_in_cycle=1000):
    print(f"From: {path} to {h5_path} for dataset:{dataset_name}")
    images_list = []
    images_paths = get_files_paths_recursive(db_path)
    i=0
    for image_path in images_paths:
        if i % 100 == 0:
            print(f"Image number: {i}")
        i += 1
        # loading images 
        pil_image = Image.open(image_path)
        np_image = np.array(pil_image).astype((np.uint8))
        images_list.append(np_image)
        # saving to h5 file
        if i % img_in_cycle == 0 and i > 0:
            # h5 dataset creation
            np_images_list = np.asarray(images_list)
            if i == img_in_cycle:
                with h5py.File(filename, "w") as hf:
                    hf.create_dataset(dataset_name, 
                                    np_images_list.shape, 
                                    data=np_images_list,
                                    maxshape=((None,)+np_images_list.shape[1:]),
                                    chunks=True)
                    hf.close()   
            # h5 dataset append
            else:
                with h5py.File(filename, "a") as hf:
                    hf[dataset_name].resize(i, axis=0)
                    hf[dataset_name][-img_in_cycle:] = np.asarray(images_list)
                    hf.close()   
            images_list.clear()

In [ ]:
convert_dataset_to_h5(path, filename, dataset_name, img_in_cycle=10)

In [ ]:
db = load_data_h5(filename, 'real')